# 02_Visualize movements for each well-plate

This script allows to show in single plot a line graph for the movement of each well. Files will be then 1) collected and zipped together and 2) visualized in a single excel file

## 1) Import packages

In [ ]:
import pandas as pd
from PIL import Image
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage
import zipfile
import os

## 2) Read the ViewPoint File

Change to pd.read_csv in case the file is csv

In [ ]:
data = pd.read_excel(r'C:\Users\Riccardo\Documents\Python\01_Project\DLLT_Tools\empirical\0_data\manual\ID480_PTX034_LMR_raw.xls')  # Replace 'your_data.csv' with your file path

## 3) Pre-processing - Create a new workbook and set column header

1) Create a list to store the individual well charts
2) Create a directory to store the images before zipping

In [ ]:
well_charts = []

wb = Workbook()
ws = wb.active

ws['A1'] = 'Well Name'
ws['B1'] = 'Image'

image_dir = 'images'
os.makedirs(image_dir, exist_ok=True)

## 4) Create the line chart and its on image

In [ ]:
wells = data['aname'].unique()
for well in wells:
    well_data = data[data['aname'] == well]
    time = well_data['end']
    dist = well_data['smldist']
    
    plt.figure()  
    plt.plot(time, dist)  
    plt.title(f'Well {well}')
    plt.xlabel('Time')
    plt.ylabel('Distance')
    plt.grid(True)  

    chart_image_path = os.path.join(image_dir, f'well_{well}.png')
    plt.savefig(chart_image_path)
    well_charts.append(chart_image_path)
    
    #This is the excel part
    ws[f'A{current_row}'] = well

    img = ExcelImage(chart_image_path)
    img.width = 200  # Adjust the image width as needed
    img.height = 150  # Adjust the image height as needed
    ws.add_image(img, f'B{current_row}')

    current_row += 1

## 4) Save the excel file plus create a zip with the images directories

In [ ]:
# Save the Excel file
wb.save('well_charts.xlsx')

# Create a zip file containing the 'images' directory
zip_file_path = 'well_charts_images.zip'
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for foldername, subfolders, filenames in os.walk(image_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            zf.write(filepath, os.path.relpath(filepath, image_dir))


print(f'Images zipped to {zip_file_path}')